In [1]:
import tempfile
import os
import sys
import subprocess
import ocgis


WD = os.path.expanduser('~/htmp/chunking-demo')
DATADIR = os.path.expanduser('~/htmp/chunking-demo-data')
GCWD = os.path.join(WD, 'chunks')
SRCNAME = os.path.join(DATADIR, 'source-grid.nc')
SOURCE_CFGRID = os.path.join(DATADIR, SRCNAME)
DESTINATION_SCRIP = os.path.expanduser('~/l/i49-ugrid-cesm/SCRIPgrid_ne16np4_nomask_c110512.nc')
DESTINATION_POINT = os.path.expanduser('~/l/i49-ugrid-cesm/SCRIPgrid_1x1pt_brazil_nomask_c110308.nc')
WEIGHT_DIR = tempfile.mkdtemp(prefix='esmf_weights')
WEIGHT_GRID = os.path.join(WEIGHT_DIR, 'esmf_weights_grid.nc')
WEIGHT_POINT = os.path.join(WEIGHT_DIR, 'esmf_weights_points.nc')
OCLI_EXE = os.path.expanduser('~/l/ocgis/src/ocli.py')

In [2]:
# This is the current command line help documentation.
out = subprocess.check_output([sys.executable, OCLI_EXE, 'chunked-rwg', '--help'])
print(out)

Usage: ocli.py chunked_rwg [OPTIONS]

  Run regridding using a spatial decomposition.

Options:
  -s, --source PATH               Path to the source grid NetCDF file.
                                  [required]
  -d, --destination PATH          Path to the destination grid NetCDF file.
                                  [required]
  -n, --nchunks_dst TEXT          Single integer or sequence defining the
                                  chunking decomposition for the destination
                                  grid. For unstructured grids, provide a
                                  single value (i.e. 100). For logically
                                  rectangular grids, two values are needed to
                                  describe the x and y decomposition (i.e.
                                  10,20).
  --merge / --no_merge            (default=True) If --merge, merge weight file
                                  chunks into a global weight file.
  -w, --weight PATH        

In [3]:
# Execute chunked conservative regridding.
cmd = ['mpirun', '-n', str(8), sys.executable, OCLI_EXE, 'chunked_rwg', '--source', SOURCE_CFGRID,
       '--destination', DESTINATION_SCRIP, '--weight', WEIGHT_GRID, '--nchunks_dst', str(5), 
       '--esmf_dst_type', 'SCRIP',
       # '--persist', '--wd', '/tmp/some_chunks'
       ]
print(' '.join(cmd) + '\n')
subprocess.check_call(cmd)
out = subprocess.check_output(['ncdump', '-h', WEIGHT_GRID])
print(out)

mpirun -n 8 /home/benkoziol/anaconda2/envs/ocgis/bin/python /home/benkoziol/l/ocgis/src/ocli.py chunked_rwg --source /home/benkoziol/htmp/chunking-demo-data/source-grid.nc --destination /home/benkoziol/l/i49-ugrid-cesm/SCRIPgrid_ne16np4_nomask_c110512.nc --weight /tmp/esmf_weightsOemqqD/esmf_weights_grid.nc --nchunks_dst 5 --esmf_dst_type SCRIP

netcdf esmf_weights_grid {
dimensions:
	n_s = 155886 ;
variables:
	int row(n_s) ;
	int col(n_s) ;
	double S(n_s) ;
}



In [4]:
# Execute regridding with a spatial subset.
cmd = ['mpirun', '-n', str(1), sys.executable, OCLI_EXE, 'chunked_rwg', '--source', SOURCE_CFGRID,
       '--destination', DESTINATION_POINT, '--weight', WEIGHT_POINT, '--esmf_dst_type', 'SCRIP', '--spatial_subset',
       # '--persist'
       ]
print(' '.join(cmd) + '\n')
subprocess.check_call(cmd)
out = subprocess.check_output(['ncdump', '-h', WEIGHT_POINT])
print(out)

mpirun -n 1 /home/benkoziol/anaconda2/envs/ocgis/bin/python /home/benkoziol/l/ocgis/src/ocli.py chunked_rwg --source /home/benkoziol/htmp/chunking-demo-data/source-grid.nc --destination /home/benkoziol/l/i49-ugrid-cesm/SCRIPgrid_1x1pt_brazil_nomask_c110308.nc --weight /tmp/esmf_weightsOemqqD/esmf_weights_points.nc --esmf_dst_type SCRIP --spatial_subset

netcdf esmf_weights_points {
dimensions:
	n_s = 6 ;
variables:
	double S(n_s) ;
	int col(n_s) ;
	int row(n_s) ;
}

